In [23]:
#Necessary Imports
from pacman import GhostRules, PacmanRules, ClassicGameRules, GameState
from game import GameStateData
from game import Game
from game import Directions
from game import Actions
from game import Agent
from util import nearestPoint
from util import manhattanDistance
import util, layout
import sys, types, time, random, os

In [24]:
# Pacman Agent, or custom defined if needed
# @ Sergey, key thing to note, you should 
# always check if a resulting action is legal
# Else just have pacman stay in place
# This basically allows the network to just keep 
# Pacman in a cubby for a bit
class SmartAgent(Agent):
    
    def getAction(self, state):
        action = Directions.WEST # Replace This with a call to the model
        if action in state.getLegalPacmanActions():
            return action
        else:
            return Directions.STOP

In [25]:
class MyGhostAgent( Agent ):
    def __init__( self, index ):
        self.index = index

    def getAction( self, state ):
        action = Directions.EAST # Replace This with a call to the model
        if action in state.getLegalActions( self.index ):
            return action
        else:
            return random.choice( state.getLegalActions( self.index ))

In [26]:
#define the ghost agent ___otherwise import it here. 
class SmartGhost( MyGhostAgent ):
    def __init__( self, index ):
        self.index = index

    def getAction( self, state ):
        action = Directions.EAST # Replace This with a call to the model
        if action in state.getLegalActions( self.index ):
            return action
        else:
            return random.choice( state.getLegalActions( self.index ))

In [27]:
args = dict()
args['layout'] = layout.getLayout("originalClassic.lay")
if args['layout'] == None: raise Exception("The layout " + options.layout + " cannot be found")
    
args['pacman'] = SmartAgent()

ghosts = [SmartGhost, MyGhostAgent,MyGhostAgent,MyGhostAgent]
args['ghosts'] = [ghosts[i](i+1) for i in range(len(ghosts))]

args['numTraining'] = 0
args['numGames'] = 100
args['record'] = True
args['catchExceptions'] = False
args['timeout'] = 30

In [28]:
#define a "run" function for the iterations
import textDisplay
def runGames( layout, pacman, ghosts, numGames, record, numTraining = 0, catchExceptions=False, timeout=30 ):
    gameDisplay = textDisplay.NullGraphics()
    rules = ClassicGameRules(timeout)
    rules.quiet = True
    games = []

    for i in range( numGames ):
        beQuiet = i < numTraining
        game = rules.newGame( layout, pacman, ghosts, gameDisplay, beQuiet, catchExceptions)
        game.run()
        if not beQuiet: games.append(game)

        if record:
            import time, cPickle
            fname = ('recorded_games/recorded-game-%d' % (i + 1)) +  '-'.join([str(t) for t in time.localtime()[1:6]])
            f = file(fname, 'w')
            components = {'layout': layout, 'actions': game.moveHistory}
            cPickle.dump(components, f)
            f.close()

    if (numGames-numTraining) > 0:
        scores = [game.state.getScore() for game in games]
        wins = [game.state.isWin() for game in games]
        winRate = wins.count(True)/ float(len(wins))
        #HNere is where you propogate the error hum
        # oh well
        
        print 'Average Score:', sum(scores) / float(len(scores))
        print 'Scores:       ', ', '.join([str(score) for score in scores])
        print 'Win Rate:      %d/%d (%.2f)' % (wins.count(True), len(wins), winRate)
        print 'Record:       ', ', '.join([ ['Loss', 'Win'][int(w)] for w in wins])

    return games

In [29]:
runGames(**args)

Pacman died! Score: -527
Pacman died! Score: -598
Pacman died! Score: -504
Pacman died! Score: -541
Pacman died! Score: -569
Pacman died! Score: -477
Pacman died! Score: -670
Pacman died! Score: -625
Pacman died! Score: -495
Pacman died! Score: -551
Pacman died! Score: -619
Pacman died! Score: -497
Pacman died! Score: -458
Pacman died! Score: -524
Pacman died! Score: -514
Pacman died! Score: -553
Pacman died! Score: -544
Pacman died! Score: -492
Pacman died! Score: -570
Pacman died! Score: -546
Pacman died! Score: -596
Pacman died! Score: -590
Pacman died! Score: -535
Pacman died! Score: -503
Pacman died! Score: -476
Pacman died! Score: -567
Pacman died! Score: -470
Pacman died! Score: -560
Pacman died! Score: -591
Pacman died! Score: -634
Pacman died! Score: -495
Pacman died! Score: -555


KeyboardInterrupt: 

In [ ]:
#Option 2: More fine grain control
# This would require me to rewrite the run method. This is ok too, just need a heads up. 